In [1]:
import sys
import numpy as np
from Bio import PDB
import pandas as pd
import seaborn as sns
import os
import pickle
import random
import torch
import esm
import re
import math
from Bio import SeqIO

from utils import old_aa, position_aa, new_aa,Esm2_embedding ,Create_mut_sequence_multiple, dataset_builder

2.4.1+cu118


In [2]:
df_m28 = pd.read_csv('../git_JANUS_DDG/Data/m28_test_set.csv')
df_m28

,Unnamed: 0,ID,MTS,DDG,DDGun,DDGun3D,Maestro,FoldX,Sequence
0,0,1btlA,A17G_E79K_M157T,-4.3000,0.5,0.0,-0.11346,-0.884510,HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
1,1,1btlA,A17S_E79K,-0.3000,-0.2,-0.1,-0.03137,-0.461080,HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
2,2,1btlA,E79K_M157T,-1.5000,0.3,0.1,-0.10197,-0.206160,HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
3,3,1fkbA,D37S_R42A,-0.1800,-0.6,-0.3,0.02025,-1.616570,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...
4,4,1fkbA,E31Q_D32N,0.4600,0.3,0.1,0.02384,0.204000,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...
5,5,1fkbA,E31Q_D32N_H94N,0.8700,0.1,0.1,-0.01712,0.241418,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...
6,6,1fkbA,I91E_A95K,-2.3600,-1.9,-1.7,-0.53359,-3.664100,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...
7,7,1fkbA,R40I_D41E,-1.4700,-0.1,-0.1,0.02639,-0.060014,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...
8,8,1fkbA,R40I_D41E_R42E_N43Y_K44E,-1.8900,-0.3,-0.3,-0.10398,-1.978490,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...
9,9,1fkbA,T21K_V23K_K47E,-2.8800,-0.9,-1.3,-0.55597,-0.768841,GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...


In [3]:
#CARICAMENTO DEL DATASET 

#IL DATASET CARICATO DEVE AVERE LE SEGUENTI COLONNE:

#ID	MTS SEQ
#ID: NOME UNIVOCO PER SEQUENZA WILD TYPE
#MTS: MUTS SEP BY '_' (position start by 1)
#Sequence: SEQUENZA LETTERE AA DELLA PROTEINA

#ESEMPIO: P29957 Q82C_A123C_.... MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...

#df = pd.read_csv('../Data/')

dataset_mutations = df_m28[['ID','MTS','DDG']].copy()
print(dataset_mutations.head())

dataset_sequences = df_m28.loc[:,['ID','Sequence']].copy()
dataset_sequences = dataset_sequences.drop_duplicates(subset='ID')
print(dataset_sequences.head())

      ID              MTS   DDG
0  1btlA  A17G_E79K_M157T -4.30
1  1btlA        A17S_E79K -0.30
2  1btlA       E79K_M157T -1.50
3  1fkbA        D37S_R42A -0.18
4  1fkbA        E31Q_D32N  0.46
       ID                                           Sequence
0   1btlA  HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
3   1fkbA  GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...
12  1hrcA  GDVEKGKKIFVQKCAQCHTVEKGGKHKTGPNLHGLFGRKTGQAPGF...
13  1jl5A  KSKTEYYNAWSEWERNAPPGNGEQREMAVSRLRDCLDRQAHELELN...
14  1jqzA  HHHHFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHI...


In [4]:
dataset_mutations['Old_AA'] = dataset_mutations.apply(old_aa, axis = 1)
dataset_mutations['Pos_AA'] = dataset_mutations.apply(position_aa, axis = 1)
dataset_mutations['New_AA'] = dataset_mutations.apply(new_aa, axis = 1)
dataset_mutations['Pos_AA'] = dataset_mutations['Pos_AA'].map(lambda x: [i-1 for i in x])

print(dataset_mutations.head())
print(dataset_sequences.head())


      ID              MTS   DDG     Old_AA         Pos_AA     New_AA
0  1btlA  A17G_E79K_M157T -4.30  [A, E, M]  [16, 78, 156]  [G, K, T]
1  1btlA        A17S_E79K -0.30     [A, E]       [16, 78]     [S, K]
2  1btlA       E79K_M157T -1.50     [E, M]      [78, 156]     [K, T]
3  1fkbA        D37S_R42A -0.18     [D, R]       [36, 41]     [S, A]
4  1fkbA        E31Q_D32N  0.46     [E, D]       [30, 31]     [Q, N]
       ID                                           Sequence
0   1btlA  HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMST...
3   1fkbA  GVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPF...
12  1hrcA  GDVEKGKKIFVQKCAQCHTVEKGGKHKTGPNLHGLFGRKTGQAPGF...
13  1jl5A  KSKTEYYNAWSEWERNAPPGNGEQREMAVSRLRDCLDRQAHELELN...
14  1jqzA  HHHHFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHI...


In [5]:
dataset_multi = dataset_builder(dataset_mutations, dataset_sequences,)

In [6]:
import pickle
with open("../DeltaDelta_BELLO/M28_test.pkl", "wb") as f:
    pickle.dump(dataset_multi, f)

In [7]:
pd.read_pickle('../DeltaDelta_BELLO/M28_test.pkl')[0]

{'id': '1btlA',
 'wild_type': array([[ 0.09218772,  0.09028463,  0.03316356, ..., -0.08108356,
          0.12128501, -0.06550556],
        [ 0.15088417,  0.09649397, -0.01178914, ..., -0.14696507,
          0.16516843,  0.03135337],
        [ 0.15183318,  0.06472991,  0.00254108, ...,  0.06931368,
          0.13081601,  0.03852017],
        ...,
        [-0.00686379,  0.1619582 ,  0.02690414, ..., -0.19595791,
         -0.21645625, -0.06292378],
        [ 0.06748256, -0.02413554, -0.12852477, ..., -0.0242437 ,
          0.05163071, -0.02304967],
        [-0.06427971,  0.09537789,  0.20633267, ..., -0.2992399 ,
         -0.06998833,  0.13745816]], dtype=float32),
 'mut_type': array([[ 0.08669748,  0.08721141,  0.01870232, ..., -0.09030939,
          0.12615316, -0.08091   ],
        [ 0.15278427,  0.10669658, -0.02356631, ..., -0.15691939,
          0.15336162,  0.01378142],
        [ 0.15871942,  0.05490726, -0.00253063, ...,  0.05961538,
          0.12243679,  0.02609443],
        ...